In [12]:
# Some potentially useful modules
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import copy


class NeuralMMAgent(object):
    '''
    Class to for Neural Net Agents that compete in the Mob task
    '''

    def __init__(self, num_in_nodes, num_hid_nodes, num_hid_layers, num_out_nodes, \
                 learning_rate=0.2, max_epoch=10000, min_sse=.01, momentum=0, \
                 creation_function=None, activation_function=None, random_seed=1):
        '''
        Arguments:
            num_in_nodes -- total # of input nodes for Neural Net
            num_hid_nodes -- total # of hidden nodes for each hidden layer
                in the Neural Net
            num_hid_layers -- total # of hidden layers for Neural Net
            num_out_nodes -- total # of output layers for Neural Net
            learning_rate -- learning rate to be used when propogating error
            max_epoch -- maximum number of epochs for our NN to run during learning
            min_sse -- minimum SSE that we will use as a stopping point
            momentum -- Momentum term used for learning
            creation_function -- function that will be used to create the
                neural network given the input
            activation_function -- list of two functions:
                1st function will be used by network to determine activation given a weighted summed input
                2nd function will be the derivative of the 1st function
            random_seed -- used to seed object random attribute.
                This ensures that we can reproduce results if wanted
        '''
        self.num_in_nodes = num_in_nodes
        self.num_hid_nodes = num_hid_nodes
        self.num_hid_layers = num_hid_layers
        self.num_out_nodes = num_out_nodes
        self.learning_rate = learning_rate
        self.max_epoch = max_epoch
        self.min_sse = min_sse
        self.momentum = momentum
        self.creation_function = creation_function
        self.activation_function = activation_function
        self.random_seed = random_seed
        self.layer_nodes = []
        self.layer_nodes.append(self.num_in_nodes)
        for i in range(self.num_hid_layers):
            self.layer_nodes.append(self.num_hid_nodes)
        self.layer_nodes.append(self.num_out_nodes)
        random.seed(self.random_seed)
        self.num_weights = [4, 2]

        assert num_in_nodes > 0 and num_hid_layers > 0 and num_hid_nodes and \
               num_out_nodes > 0, "Illegal number of input, hidden, or output layers!"

    def train_net_incremental(self, input_list, output_list, max_num_epoch=100000, min_sse=0.001):
        ''' Trains neural net using incremental learning
            (update once per input-output pair)
            Arguments:
                input_list -- 2D list of inputs
                output_list -- 2D list of outputs matching inputs
            Outputs:
                1d list of errors (total error each epoch) (e.g., [0.1])
        '''
        # Some code...#
        # all_err.append(total_err)

        # if (total_err < min_sse):
        # break
        all_err = []
        errors = 0
        for epoch in range(max_num_epoch):
            total_err = 0
            for row in range(len(input_list)):
                activations = self._feed_forward(input_list, row)
                # errors.append([output_list[row][0] - activations[-1][0]])
                # self._adjust_weights_bias(activations, error)
                errors = output_list[row][0] - activations[-1][0]
                result_deltas = self._calculate_deltas(activations, errors, prev_weight_deltas=None)
                self._adjust_weights_bias(result_deltas[1])
                total_err += output_list[row][0] - activations[-1][0]

            all_err.append(total_err)
            if total_err < min_sse:
                break
        return all_err

    def _feed_forward(self, input_list, row):
        '''Used to feedforward input and calculate all activation values
            Arguments:
                input_list -- a list of possible input values
                row -- the row from that input_list that we should use
            Outputs:
                list of activation values
        '''
        
        activations = []
        weight_multiply = []
        hid_activation = []
        output = 0

        # Input layer activations
        activations.append(input_list[row])

        # Hidden layer activations
        start = 0
        end = self.num_hid_nodes
        for i in activations[0]:
            for j in range(start, end):
                weight_multiply.append(i * self.weights[0][j])
            start = end
            end = end + self.num_hid_nodes
        for j in range(self.num_hid_nodes):
            sum_wx = sum([activations[i][0] * self.weights[i + 1][j] for i in range(self.num_hid_layers)])
            hid_activation.append(sum_wx)
        activations.append(hid_activation)

        # Output layer activation
        for i in range(self.num_out_nodes):
            sum_wx = sum(
                [hid_activation[j] * self.weights[-1][j * self.num_out_nodes + i] for j in range(self.num_hid_nodes)])
            output += sum_wx
        activations.append([output])

        return activations

    def _calculate_deltas(self, activations, errors, prev_weight_deltas=None):
        '''Used to calculate all weight deltas for our neural net
            Parameters:
                activations -- a 2d list of activation values
                errors -- a 2d list of errors
                prev_weight_deltas [OPTIONAL] -- a 2d list of previous weight deltas
            Output:
                A tuple made up of 3 items:
                    A 2d list of little deltas (e.g., [[0, 0], [-0.1, 0.1], [0.1]])
                    A 2d list of weight deltas (e.g., [[-0.1, 0.1, -0.1, 0.1], [0.1, 0.1]])
                    A 2d list of bias deltas (e.g., [[0, 0], [-0.1, 0.1], [0]])
        '''

        # Calculate error gradient for each output node & propgate error
        #   (calculate weight deltas going backward from output_nodes)
        little_deltas = []
        little_deltas.append([0] * self.num_in_nodes)
        little_deltas.append([0] * self.num_hid_nodes)
        little_deltas.append([0] * self.num_out_nodes)
        error_gradient = 0

        # output layer
        for i in range(self.num_out_nodes):
            error_gradient = errors * self.sigmoid_af_deriv(activations[-1][i])
            little_deltas[-1][i] = error_gradient

        # hidden-layer to output layer
        error_gradient = 0
        for j in range(self.num_hid_nodes):
            error_gradient += little_deltas[-1][0] * self.weights[-1][j]
            error_gradient *= self.sigmoid_af_deriv(activations[-1][0])
            little_deltas[-2][j] = error_gradient

        # middle hidden layer
        start = 0
        end = self.num_hid_nodes
        if self.num_hid_layers == 1:
            pass
        else:
            for i in range(self.num_hid_layers, -1, -1):
                for j in range(self.num_hid_nodes):
                    error_gradient = 0
                    for t in range(self.num_hid_layers, -1, -1):
                        for k in range(start, end):
                            error_gradient += little_deltas[i][j] * self.weights[t][k]
                        start = self.num_hid_nodes
                        end = start + self.num_hid_nodes
                        error_gradient *= self.sigmoid_af_deriv(activations[i][j])
                        little_deltas[i - 1][j] = error_gradient

        for i in range(self.num_in_nodes):
            little_deltas[0][i] = 0

        weight_deltas = []
        for i in range(self.num_hid_layers):
            weight_deltas.append(np.zeros((self.num_hid_nodes * self.num_hid_nodes, 1)))
        weight_deltas.append(np.zeros((self.num_hid_nodes * self.num_out_nodes, 1)))

        start_w = 0
        end_w = self.num_hid_nodes
        for i in range(self.num_hid_layers):
            for j in range(self.num_hid_nodes):
                for k in range(start_w,end_w):
                    weight_deltas[i][k] = self.learning_rate * little_deltas[i+1][j] * activations[i+1][j]
                start_w = self.num_hid_nodes
                end_w = start_w + self.num_hid_nodes

        for i in range(self.num_hid_nodes * self.num_out_nodes):
            weight_deltas[-1][i] = self.learning_rate * little_deltas[-1][0] * activations[-1][0]


        """           
        bias_deltas = []
        for i in range(self.num_hid_layers):
            bias_deltas.append([0]*self.layer_nodes[i])

        for i in range(self.num_layers-1):
            for j in range(self.layer_nodes[i+1]):
                bias_deltas[i+1][j] = self.learning_rate * little_deltas[i+1][j] + \
                                            self.momentum * self.previous_bias_deltas[i+1][j]
        """
        return (little_deltas, weight_deltas)

    def _adjust_weights_bias(self, weight_deltas):
        '''Used to apply deltas
        Parameters:
            weight_deltas -- 2d list of weight deltas
            bias_deltas -- 2d list of bias deltas
        Outputs:
            A tuple w/ the following items (in order):
            2d list of all weights after updating (e.g. [[-0.071, 0.078, 0.313, 0.323], [-0.34, 0.021]])
            list of all biases after updating (e.g., [[0, 0], [0, 0], [0]])
        '''
        for i in range(self.num_hid_layers + 1):
            for j in range(self.num_weights[i]):
                self.weights[i][j] -= weight_deltas[i][j]

        """
        for i in range(self.num_layers):
            self.bias[i] -= self.bias_deltas[i]
        """
        # self.previous_weight_deltas = weight_deltas
        # self.previous_bias_deltas = self.bias_deltas

        return self.weights

    #########ACCESSORS

    def get_weights(self):
        return (self.weights)

    def set_weights(self, weights):
        self.weights = weights

    def get_bias(self):
        return (self.bias)

    def set_biases(self, bias):
        self.bias = bias

    ################

    @staticmethod
    def sigmoid_af(summed_input):
        # Sigmoid function
        return 1 / (1 + np.exp(-summed_input))
        pass

    @staticmethod
    def sigmoid_af_deriv(sig_output):
        # the derivative of the sigmoid function
        return sig_output * (1 - sig_output)
        pass


# ----#
# Some quick test code

test_agent = NeuralMMAgent(2, 2, 1, 1, random_seed=5, max_epoch=1000000, learning_rate=0.2, momentum=0)
test_in = [[1, 0], [0, 0], [1, 1], [0, 1]]
test_out = [[1], [0], [0], [1]]
test_agent.set_weights([[-.37, .26, .1, -.24], [-.01, -.05]])
test_agent.set_biases([[0, 0], [0, 0], [0]])
all_errors = test_agent.train_net_incremental(test_in, test_out, min_sse=test_agent.min_sse,
                                              max_num_epoch=test_agent.max_epoch)






In [14]:
all_errors[-1][0]

1.0271711954963474